#Hive intermediate managed tables environment

In [0]:
%sql
create database if not exists intermed

In [0]:
%sql
create or replace table intermed.assets
(
    asset_id string
    ,asset_name string
    ,flag_is_crypto tinyint
)
using delta
location '/mnt/intermed/hive/assets'
comment 'inserting assets intermed into hive managed table'

In [0]:
%sql
insert into intermed.assets
select
    a.asset_id as asset_id
    ,a.name as asset_name
    ,a.type_is_crypto as flag_is_crypto
from delta.`dbfs:/mnt/inbound/hive/assets/` a
group by 1, 2, 3

num_affected_rows,num_inserted_rows
18571,18571


In [0]:
%sql
create or replace table intermed.exchanges
(
    exch_id string
    ,exch_website string
    ,exch_name string
)
using delta
location '/mnt/intermed/hive/exchanges'
comment 'inserting exchanges intermed into hive managed table'

In [0]:
%sql
insert into intermed.exchanges
select 
    e.exchange_id as exch_id
    ,e.website as exch_website
    ,e.name as exch_name
from delta.`dbfs:/mnt/inbound/hive/exchanges/` e
group by 1, 2, 3

num_affected_rows,num_inserted_rows
356,356


In [0]:
%sql
create or replace table intermed.symbols
(
    symb_id string
    ,exch_id string
    ,symb_type string
    ,asset_id string
    ,asset_id_quote string
)
using delta
location '/mnt/intermed/hive/symbols'
comment 'inserting symbols intermed into hive managed table'

In [0]:
%sql
insert into intermed.symbols
select 
    s.symbol_id as symb_id
    ,s.exchange_id as exch_id
    ,s.symbol_type as symb_type
    ,s.asset_id_base as asset_id
    ,s.asset_id_quote as asset_id_quote
from delta.`dbfs:/mnt/inbound/hive/symbols/` s
group by 1, 2, 3, 4, 5

num_affected_rows,num_inserted_rows
578795,578795


In [0]:
%sql
create or replace table intermed.tradeslatest
(
    symb_id string
    ,tmstp_exch timestamp
    ,tmpstp_coinapi timestamp
    ,transaction_id string
    ,price double
    ,size double
    ,taker_side_trade_type string
    ,exchge_side_trade_type string
)
using delta
location '/mnt/intermed/hive/tradeslatest'
comment 'inserting tradeslatest intermed into hive managed table'

In [0]:
%sql
insert into intermed.tradeslatest
select
  t.symbol_id as symb_id
  ,t.time_exchange as tmstp_exch
  ,t.time_coinapi as tmpstp_coinapi
  ,t.uuid as transaction_id
  ,coalesce(round(avg(t.price),8),0) as price
  ,coalesce(round(sum(t.size),8),0) as size
  ,initcap(t.taker_side) as taker_side_trade_type
  ,case
    when t.taker_side = 'BUY' then 'Sell'
    when t.taker_side = 'SELL' then 'Buy'
    when t.taker_side = 'BUY_ESTIMATED' then 'Sell_estimated'
    when t.taker_side = 'SELL_ESTIMATED' then 'Buy_estimated'
    else 'Unknown'
  end as exchge_side_trade_type
from delta.`dbfs:/mnt/inbound/hive/tradeslatest/` t
group by 1, 2, 3, 4, 7, 8

num_affected_rows,num_inserted_rows
50,50
